In [2]:
import os, subprocess, time, atexit, signal

# 1) runtime dir（避免你之前的 XDG 报错）
os.environ["XDG_RUNTIME_DIR"] = "/tmp/xdg"
os.makedirs("/tmp/xdg", exist_ok=True)
os.chmod("/tmp/xdg", 0o700)

# 2) 强制走 Xvfb 的 Linux64（禁止 CloudRendering 路径）
os.environ["AI2THOR_PLATFORM"] = "Linux64"

# 3) 启动虚拟显示
os.environ["DISPLAY"] = ":99"

# 如果之前残留 Xvfb，占用 :99，会启动失败；先粗暴清一下（可选）
subprocess.run("pkill -f 'Xvfb :99' || true", shell=True)

proc = subprocess.Popen([
    "Xvfb", ":99",
    "-screen", "0", "1280x1024x24",
    "-ac", "+extension", "GLX", "+render", "-noreset"
])

time.sleep(0.5)

def cleanup():
    if proc.poll() is None:
        proc.send_signal(signal.SIGTERM)

atexit.register(cleanup)

print("Xvfb OK, DISPLAY=", os.environ["DISPLAY"], "AI2THOR_PLATFORM=", os.environ["AI2THOR_PLATFORM"])
from ai2thor.controller import Controller

# 关键：这里也显式写 platform，双保险
controller = Controller(platform="Linux64", headless=False)
print("Controller created.")

Xvfb OK, DISPLAY= :99 AI2THOR_PLATFORM= Linux64
Controller created.


In [3]:
import sys
import os
import random
import time
import json
import traceback

# --- 1. 路径设置 ---
# 假设你的 Notebook 文件在 Neural-TAMP 根目录下运行
# 如果在子目录，请调整这里的路径，确保能 import src
project_root = os.getcwd() 
if project_root not in sys.path:
    sys.path.append(project_root)

# --- 2. 导入核心模块 ---
from src.env.procthor_wrapper import ProcTHOREnv
from src.perception.oracle_interface import OracleInterface
from src.memory.graph_manager import GraphManager
from src.task.task_generator import TaskGenerator 

print(f"📂 Project Root: {project_root}")

📂 Project Root: /workspace/Neural-TAMP


In [4]:
print("="*60)
print("🛠️  Initializing Neural-TAMP System (Run Once)")
print("="*60)

try:
    print("[Init] Loading ProcTHOR Environment (This may take a while)...")
    
    # 耗时操作：启动 Unity 和 X Server
    # 注意：这里严格保持无参数，与 main.py 一致
    env = ProcTHOREnv() 
    
    # 初始化其他模块
    oracle = OracleInterface(env)
    memory = GraphManager(save_dir="Neural-TAMP/debug_data")
    
    # 实例化生成器
    # (如果你修改了 TaskGenerator 代码，需要重新运行这个 Cell 或者用 reload 重新加载它)
    task_gen = TaskGenerator()
    
    print("\n✅ System Ready! Environment is live.")
    
except Exception as e:
    print(f"\n❌ Init Failed: {e}")
    traceback.print_exc()

🛠️  Initializing Neural-TAMP System (Run Once)
[Init] Loading ProcTHOR Environment (This may take a while)...
[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 5885.59it/s]


[Env] Initializing Controller...

❌ Init Failed: Error encountered when running action {'action': 'Initialize', 'agentMode': 'default', 'visibilityDistance': 5.0, 'gridSize': 0.25, 'renderDepthImage': True, 'renderInstanceSegmentation': False, 'fieldOfView': 90, 'sequenceId': 1} in scene FloorPlan10_physics.


Traceback (most recent call last):
  File "/workspace/tamp_env/lib/python3.10/site-packages/ai2thor/controller.py", line 979, in step
    self.last_event = self.server.receive()
  File "/workspace/tamp_env/lib/python3.10/site-packages/ai2thor/fifo_server.py", line 229, in receive
    metadata, files = self._recv_message(
  File "/workspace/tamp_env/lib/python3.10/site-packages/ai2thor/fifo_server.py", line 145, in _recv_message
    header = self._read_with_timeout(
  File "/workspace/tamp_env/lib/python3.10/site-packages/ai2thor/fifo_server.py", line 134, in _read_with_timeout
    raise TimeoutError(f"Reading from AI2-THOR backend timed out (using {timeout}s) timeout.")
TimeoutError: Reading from AI2-THOR backend timed out (using 100.0s) timeout.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_2846/363243541.py", line 10, in <module>
    env = ProcTHOREnv()
  File "/workspace/Neural-TAMP/src/env/procthor_wr